In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import matplotlib.pyplot as plt 
import torchvision
import sys
sys.path.append("..")
import transformations as custom_transforms
import datasets as datasets

# do not use transorms.ToTensor(),already implemented implicitly. 

L = datasets.SegmentationDatasetLabeled(
    dir_images="./my_dataset/images/",
    dir_masks="./my_dataset/masks/",
    
)

gaussian_blur = custom_transforms.GaussianBlur(9,1000)
random_affine = custom_transforms.RandomAffine(
    degrees=50,
    translate=(0.05, 0.05),
    scale=(0.95, 1.05)
)
random_prespective = custom_transforms.RandomPerspective(p=1)




transforamtions = [gaussian_blur,random_affine,random_prespective]
t_choice = torchvision.transforms.RandomChoice(transforamtions)
t_order = custom_transforms.RandomApply(transforamtions)
for i in range(len(L)):
    img_, mask_ = L[i]
    mask_ = mask_[:,:,:]
    img_, mask_ = t_order(img_,mask_)
    
    print(img_.size())
    print(mask_.size())

    img__ = img_.permute(1, 2, 0)
    mask__ = mask_.permute(1, 2, 0)
    plt.imshow(img__)
    plt.figure()
    plt.imshow(mask__)
    plt.show()

/datagrid/personal/hruskan1/miniconda3/envs/gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'torchvision.transforms' has no attribute 'RandomTransforms'